<a href="https://colab.research.google.com/github/chandalaniharika279/Algorithm-And-Analysis/blob/EdmondBlossom/EdmondBlossom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class Node:
    index = 0

    def __init__(self):
        self.neighbors = []
        self.is_visited = False
        self.parent = None
        self.mate = None
        self.index = Node.index
        Node.index += 1

    def __repr__(self):
        return str(self.index)

class SuperNode(Node):
    def __init__(self):
        super().__init__()
        self.subnodes = []
        self.original_edges = []

class Path:
    def __init__(self):
        self.nodes = []

    def replace(self, snode):
        index = self.nodes.index(snode)
        cur_node = self.nodes[index - 1]
        for edge in snode.original_edges:
            if edge[0] == cur_node:
                cur_node = edge[1]
            elif edge[1] == cur_node:
                cur_node = edge[0]

        while cur_node.parent != snode.parent:
            self.nodes.insert(index, cur_node)
            self.nodes.insert(index + 1, cur_node.mate)
            index += 2
            cur_node = cur_node.parent

        self.nodes.append(cur_node)

class Match:
    def __init__(self, nodes):
        self.nodes = nodes
        self.freenodes = list(nodes)
        for node in nodes:
            self.freenodes.append(node)
        self.supernodes = []

    @staticmethod
    def from_edges(N, edges):
        nodes = [Node() for _ in range(N)]
        for i, j in edges:
            nodes[i].neighbors.append(nodes[j])
            nodes[j].neighbors.append(nodes[i])
        return Match(nodes)

    def clear_nodes(self):
        for node in self.nodes:
            node.is_visited = False
            node.parent = None

    def find_augmenting_path(self, root):
        self.clear_nodes()
        queue = [root]
        while queue:
            cur_node = queue.pop(0)
            cur_node.is_visited = True
            for neighbor in cur_node.neighbors:
                if neighbor == cur_node.parent:
                    continue
                if neighbor.is_visited:
                    cycle = self.find_cycles(neighbor, cur_node)
                    if len(cycle) % 2 == 1:
                        snode = self.shrink_blossom(cycle)
                        self.supernodes.append(snode)
                        for v in cycle:
                            if v in queue:
                                queue.remove(v)
                        snode.is_visited = True
                        queue.insert(0, snode)
                        break
                elif neighbor.mate is None:
                    neighbor.parent = cur_node
                    return self.construct_augmenting_path(neighbor)
                elif neighbor.mate != cur_node:
                    neighbor.is_visited = True
                    neighbor.mate.is_visited = True
                    neighbor.parent = cur_node
                    neighbor.mate.parent = neighbor
                    queue.append(neighbor.mate)
        raise Exception('Cannot find an augmenting path')

    def maximum_matching(self):
        while self.freenodes:
            for node in self.freenodes:
                try:
                    path = self.find_augmenting_path(node)
                    self.invert_path(path)
                    self.freenodes.remove(path.nodes[0])
                    self.freenodes.remove(path.nodes[-1])
                    break
                except Exception:
                    pass
            else:
                break

    def invert_path(self, path):
        for i in range(0, len(path.nodes), 2):
            path.nodes[i].mate = path.nodes[i + 1]
            path.nodes[i + 1].mate = path.nodes[i]

    def construct_augmenting_path(self, node):
        path = Path()
        path.nodes.append(node)
        while node.parent is not None:
            node = node.parent
            path.nodes.append(node)

        while self.supernodes:
            snode = self.supernodes.pop()
            self.expand_supernode(snode)
            path.replace(snode)

        return path

    def find_cycles(self, node1, node2):
        def find_ancestors(node):
            ancestors = []
            visited = set()
            while node is not None:
                if node in visited:
                    break
                visited.add(node)
                ancestors.append(node)
                node = node.parent
            return ancestors

        ancestors1 = find_ancestors(node1)
        ancestors2 = find_ancestors(node2)

        # Find common ancestor
        i, j = len(ancestors1) - 1, len(ancestors2) - 1
        while i >= 0 and j >= 0 and ancestors1[i] == ancestors2[j]:
            i -= 1
            j -= 1

        cycle = ancestors1[:i + 1] + ancestors2[j + 1::-1]
        return cycle

    def shrink_blossom(self, blossom):
        snode = SuperNode()
        for node in blossom:
            snode.subnodes.append(node)
            for adj_node in node.neighbors:
                if adj_node not in blossom:
                    snode.original_edges.append((node, adj_node))
                if adj_node.parent in blossom:
                    adj_node.parent = snode

        for node1, node2 in snode.original_edges:
            node1.neighbors.remove(node2)
            node2.neighbors.remove(node1)
            node2.neighbors.append(snode)
            snode.neighbors.append(node2)

        return snode

    def expand_supernode(self, snode):
        for node1, node2 in snode.original_edges:
            node1.neighbors.append(node2)
            node2.neighbors.append(node1)
            node2.neighbors.remove(snode)
            snode.neighbors.remove(node2)


edges = [(0, 2), (0, 8), (1, 3), (1, 8), (2, 4), (2, 5), (3, 4), (4, 5), (5, 6), (6, 7), (8, 1), (9, 1)]
match = Match.from_edges(12, edges)
match.maximum_matching()

for node in match.nodes:
    if node.mate:
        print(f'Node {node} is matched with Node {node.mate}')

Node 0 is matched with Node 2
Node 1 is matched with Node 9
Node 2 is matched with Node 0
Node 3 is matched with Node 1
Node 4 is matched with Node 5
Node 5 is matched with Node 4
Node 6 is matched with Node 7
Node 7 is matched with Node 6
Node 9 is matched with Node 1
